### Simulated annealing

Z uporabo metahevrističnega algoritma preverjamo našo hipotezo o grafih z največjo sigma nepravilnostjo.

Opis algoritma:

- Glej pdf iz spletne - stran 6
- Soseščina bo predstavljena kot grafi, kjer vozlišču odvzamemo povezavo in do dve povezavi dodamo. 
- Ohlajanje bo po principu: T(k+1) = \alpha * T(k), \alpha in (0,1)
- Verjetnost izbire slabše možnosti bo izračunana z Boltzmanovo porazdelitvijo

In [7]:
# importing necessary libraries
import random
import math
from itertools import combinations
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# function for generating random neighbour of graph G
def random_neighbour(G):
    # get node
    N = G.vertices()[randrange(G.order())]

    # remove one edge of node N
    if len(G.neighbors(N)) != 0:
        V = G.neighbors(N)[randrange(len(list(G.neighbors(N))))]
        G.delete_edge(N, V)
    
    # add up to two edges
    k = 0 # max number of iterations counter
    # i is 0, 1 or 2, depending on probability (0: .4, 1: .45, 2: .15)
    i = 0
    if random.random() > .4:
        i += 1
        if random.random() > .75:
            i += 1

    j = 0 # counter for number of added edges

    while k < 8 and j < i:
        k += 1
        # new node
        V = G.vertices()[randrange(G.order())]
        if V == N or V in G.neighbors(N): # if node is N or is already connected to N, we skip it
            continue
        G.add_edge(N, V)
    
        # is triangle free
        if G.triangles_count() != 0:
            G.delete_edge(N, V) # if its not triangle free, we remove the edge that we added
        else: 
            j += 1

            continue

    return G # we return random neighbour

In [8]:
# sigma total irr, our f(s)
def sigma_irr(G):
    return sum((u - v) ** 2 for u, v in combinations(G.degree(), 2))

In [9]:
import time
# simulated annealing - main function (our initial solution, initial temperature, max. it., factor for T. decay)
def simulated_annealing(initial_G, T_0, max_iterations_without_improvement, alpha):
    G = initial_G.copy() # set G
    Best = G.copy() # set current best
    Best_sigma = sigma_irr(Best) # save best sigma value
    T = T_0 # set initial temperature
    num_of_it_without_improvement = 0 # counter for iterations without improvement
    current_sigma_total_irr = Best_sigma  # sigma value for current graph
    
    while num_of_it_without_improvement < max_iterations_without_improvement:
        
        G_mod = random_neighbour(G) # new solution
        G_mod_sigma = sigma_irr(G_mod) # calculate sigma of new solution
        # check, whether its better
        if G_mod_sigma > current_sigma_total_irr:
            G = G_mod # if it's better, set as current
            current_sigma_total_irr = G_mod_sigma # update current sigma value
            num_of_it_without_improvement = 0 # reset counter for i.w.i.

        # choose worse option nonetheless with certain probability
        else:
            num_of_it_without_improvement += 1 # we didn't have improvement

            # get probability with Boltzmann distribution
            p = math.exp(-(current_sigma_total_irr - G_mod_sigma) / T)

            
            if random.random() < p: # accept change
                G = G_mod
                current_sigma_total_irr = G_mod_sigma
        
        # if its better, save as best
        if current_sigma_total_irr > Best_sigma:
            Best = G.copy()
            Best_sigma = current_sigma_total_irr
        
        # update Temperature
        T = alpha * T
    
    return Best_sigma, Best, G # return best sigma, best solution and last option

### Pridobivanje prve rešitve

In [10]:
# generate star graph with max sigma total irr. (in center there can be multiple vertices)
def generate_max_sigma_star_graph(n):
    
    # generate star graph with or order n and c central vertices
    def generate_star_graph(n, c):
        graph = {} # dict. of neighbours
        i = 0
        for k in range(n): 
            i += 1
            if i <= c: # if vertex is central, set outer vertices as neighbors
                graph[k] = list(range(c, n))
            else: # else vertex is outer, set central vertices as neighbors
                graph[k] = list(range(c))
        
        return Graph(graph)


    max_sigma = 0 # for keeping maximum sigma value
    max_sigma_graphs = [] # list of graphs with maximum sigma value

    # check all graphs, where number of central nodes varies (we assume the number of central nodes from theory)
    for c in range(round(n * 0.14), round(n * 0.16)):
        G = generate_star_graph(n, c) # get star graph
        sigma = sigma_irr(G) # get sigma value of star graph

        if sigma > max_sigma: # update if its the best
            max_sigma_graphs = [G]
            max_sigma = sigma
        elif sigma == max_sigma: # if equal, just append
            max_sigma_graphs.append(G)
    
    return max_sigma_graphs # return all graphs with max sigma value

### Pregledovanje okolice

In [11]:
def raziskovanje_okolice(n, T_0, max_it, alpha, s = 20, r = 100):

    # check if there is better solution for any orded
    found_better_anywhere = False

    for j in range(s, n+1):
        found = False # check for better solution in this order
        graphs_found = [] # keep count of better graphs
        start = time.time()
        # make initial graph
        initial_G = generate_max_sigma_star_graph(j)[0]
        max_sigma = sigma_irr(initial_G)
    
        # run metaheuristic method 100 or r times
        for i in range(r):
            best_sigma, Best, G = simulated_annealing(initial_G, T_0, max_it, alpha)
            if best_sigma > max_sigma:
                found = True
                graphs_found.append(Best)
                

        # print if better solution is found for thist order
        if found == True:
            found_better_anywhere = True
            print(f"{j} - FOUND BETTER!")
        else:
            print(f"{j} -False")
    
    # print for all orders
    if found_better_anywhere == True:
        print("FOUND BETTER OPTION SOMEWHERE")
    else:
        print("THEORY HOLDS")

In [ ]:
n = 500 # check graphs up to order n
alpha = .7 # cooling koeficient
T_0 = 1000 # initial temperature
max_it = 5 # number of iterations without improvement
raziskovanje_okolice(n, T_0, max_it, alpha)

In [13]:
r = 5 # number of algorithm iterations
alpha = .7 # cooling koeficient
T_0 = 1000 # initial temperature
max_it = 2 # number of iterations without improvement
n = 1100 # check graphs up to order n
s = 1000 # check graphs from order s
raziskovanje_okolice(n, T_0, max_it, alpha, s, r)

n = 2050 # check graphs up to order n
s = 2000 # check graphs from order s
raziskovanje_okolice(n, T_0, max_it, alpha, s, r)

n = 3025 # check graphs up to order n
s = 3000 # check graphs from order s
raziskovanje_okolice(n, T_0, max_it, alpha, s, r)

n = 4010 # check graphs up to order n
s = 4000 # check graphs from order s
raziskovanje_okolice(n, T_0, max_it, alpha, s, r)

n = 5005 # check graphs up to order n
s = 5000 # check graphs from order s
raziskovanje_okolice(n, T_0, max_it, alpha, s, r)

1000 -False
1001 -False
1002 -False
1003 -False
1004 -False
1005 -False
1006 -False
1007 -False
1008 -False
1009 -False
1010 -False
1011 -False
1012 -False
1013 -False
1014 -False
1015 -False
1016 -False
1017 -False
1018 -False
1019 -False
1020 -False
1021 -False
1022 -False
1023 -False
1024 -False
1025 -False
1026 -False
1027 -False
1028 -False
1029 -False
1030 -False
1031 -False
1032 -False
1033 -False
1034 -False
1035 -False
1036 -False
1037 -False
1038 -False
1039 -False
1040 -False
1041 -False
1042 -False
1043 -False
1044 -False
1045 -False
1046 -False
1047 -False
1048 -False
1049 -False
1050 -False
1051 -False
1052 -False
1053 -False
1054 -False
1055 -False
1056 -False
1057 -False
1058 -False
1059 -False
1060 -False
1061 -False
1062 -False
1063 -False
1064 -False
1065 -False
1066 -False
1067 -False
1068 -False
1069 -False
1070 -False
1071 -False
1072 -False
1073 -False
1074 -False
1075 -False
1076 -False
1077 -False
1078 -False
1079 -False
1080 -False
1081 -False
1082 -False
1083